In [1]:
import pulp

In [2]:
model_1_a = pulp.LpProblem("Manufacturing_company", pulp.LpMinimize)
products_list = ["A", "B", "C"]
months_list = ["January", "February", "March", "April"]
x_var = pulp.LpVariable.dicts(f"Monthly_production", [(i,j) for i in products_list for j in months_list], lowBound=0, cat=pulp.LpInteger)
# Time constraints
hours_per_unit = {"A":1.2, "B":1.3, "C":1}
monthly_available_hours = [500, 450, 400, 350]
for i in range(len(months_list)):
    model_1_a += hours_per_unit["A"] * pulp.lpSum(x_var["A", months_list[i]]) \
                + hours_per_unit["B"] * pulp.lpSum(x_var["B", months_list[i]]) \
                + hours_per_unit["C"] * pulp.lpSum(x_var["C", months_list[i]]) \
        <= monthly_available_hours[i], f"Avaiable_time_in_{months_list[i]}"
# Demand constraints
monthly_demand_matrix = [
    [100, 110, 120, 130],# demand of A in four months
    [80, 90, 100, 110],
    [120, 130, 140, 150]
]

# Meet January demand
Storage_in_January = {"A":0, "B":0, "C":0}
for i in range(len(products_list)):
    model_1_a += x_var[products_list[i], "January"]>= monthly_demand_matrix[i][0], f"Demand for {products_list[i]} in January"
    Storage_in_January[products_list[i]] += x_var[products_list[i], "January"] - monthly_demand_matrix[i][0]
print(Storage_in_January)
# Meet Feburary demand
Storage_in_Feburary = {"A":0, "B":0, "C":0}
for i in range(len(products_list)):
    model_1_a += x_var[products_list[i], "February"] + Storage_in_January[products_list[i]] >= monthly_demand_matrix[i][1], f"Demand for {products_list[i]} in February"
    Storage_in_Feburary[products_list[i]] += x_var[products_list[i], "February"] + Storage_in_January[products_list[i]] - monthly_demand_matrix[i][1]
print(Storage_in_Feburary)
# Meet March demand
Storage_in_March = {"A":0, "B":0, "C":0}
for i in range(len(products_list)):
    model_1_a += x_var[products_list[i], "March"] + Storage_in_Feburary[products_list[i]] >= monthly_demand_matrix[i][2], f"Demand for {products_list[i]} in March"
    Storage_in_March[products_list[i]] += x_var[products_list[i], "March"] + Storage_in_Feburary[products_list[i]] - monthly_demand_matrix[i][2] 
print(Storage_in_March)
# Meet April demand
Storage_in_April = {"A":0, "B":0, "C":0}
for i in range(len(products_list)):
    model_1_a += x_var[products_list[i], "April"] + Storage_in_March[products_list[i]] >= monthly_demand_matrix[i][3], f"Demand for {products_list[i]} in April"
    Storage_in_April[products_list[i]] += x_var[products_list[i], "April"] + Storage_in_March[products_list[i]] - monthly_demand_matrix[i][3] 
print(Storage_in_April)

# Holding cost
merged_storage = Storage_in_January.copy() 
merged_storage.update(Storage_in_Feburary)
merged_storage.update(Storage_in_March)
merged_storage.update(Storage_in_April)
InventoryCost_in_fourPeriods = 1*merged_storage["A"] + 2*merged_storage["B"] + 0.5*merged_storage["C"]

# Overall production cost
months_list_exclued_March = ["January", "February", "April"]
# Increased production cost of B in March 
ProductionCost = 20*pulp.lpSum(x_var["A", month_] for month_ in months_list)\
                + 30*pulp.lpSum(x_var["B", month_] for month_ in months_list_exclued_March)\
                + 35*x_var["B", "March"]\
                + 10*pulp.lpSum(x_var["C", month_] for month_ in months_list)
# Objective
model_1_a += ProductionCost + InventoryCost_in_fourPeriods, "Objective"

model_1_a.solve()
print("Model Status:{}".format(pulp.LpStatus[model_1_a.status]))
for v in model_1_a.variables():
	  print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)
print("Objective=", pulp.value(model_1_a.objective))

{'A': 1*Monthly_production_('A',_'January') + -100, 'B': 1*Monthly_production_('B',_'January') + -80, 'C': 1*Monthly_production_('C',_'January') + -120}
{'A': 1*Monthly_production_('A',_'February') + 1*Monthly_production_('A',_'January') + -210, 'B': 1*Monthly_production_('B',_'February') + 1*Monthly_production_('B',_'January') + -170, 'C': 1*Monthly_production_('C',_'February') + 1*Monthly_production_('C',_'January') + -250}
{'A': 1*Monthly_production_('A',_'February') + 1*Monthly_production_('A',_'January') + 1*Monthly_production_('A',_'March') + -330, 'B': 1*Monthly_production_('B',_'February') + 1*Monthly_production_('B',_'January') + 1*Monthly_production_('B',_'March') + -270, 'C': 1*Monthly_production_('C',_'February') + 1*Monthly_production_('C',_'January') + 1*Monthly_production_('C',_'March') + -390}
{'A': 1*Monthly_production_('A',_'April') + 1*Monthly_production_('A',_'February') + 1*Monthly_production_('A',_'January') + 1*Monthly_production_('A',_'March') + -460, 'B': 1*Mon